# BFS and DFS used to determine a vertex ordering with the minimum cost

# Vertices and possible parent sets for each vertex are provided in a file, and each parent set has an associated cost.

In [1]:
import copy

filename = input("Enter the file name: ")
# Reading lines from file
with open(filename) as f:
    lines = f.readlines()

Enter the file name: data0.txt


In [2]:
# To split a line and extract vertex, parent, cost from it
def splitting(line):
    i = 0
    word = ""
    # str[0] will have vertex, str[1] will have parent set and str[2] will have cost
    str = [-1, -1, -1]
    commas = 0
    while line[i] != '\n':
        if line[i] == ',' and commas == 0:
            str[0] = word
            word = ""
            commas += 1
            i += 1
        if line[i] == "}":
            word += line[i]
            str[1] = word
            word = ""
            i += 2
            commas += 1
        word += line[i]
        i += 1
    str[2] = word
    return str

In [3]:
class Vertex:
    def __init__(self, v, p, c):
        self.vertex = v
        self.parent = p
        self.parent_cost = c

In [4]:
vertices = []  # Stores the vertices

# Storing vertices
for line in lines:
    str = splitting(line)
    var = str[0]
    if var not in vertices:
        vertices.append(var)

# Storing parent set and cost
i = 0
temp_list1 = []
temp_list2 = []
my_Vertices = []
for line in lines:
    str = splitting(line)
    if str[0] == vertices[i]:
        temp_list1.append(str[1])
        var = str[2].strip('\n')
        num = float(var)
        temp_list2.append(num)
    else:
        my_Vertices.append(Vertex(vertices[i], temp_list1, temp_list2))
        temp_list1 = []
        temp_list2 = []
        temp_list1.append(str[1])
        var = str[2].strip('\n')
        num = float(var)
        temp_list2.append(num)
        i += 1
# Appending the last vertex data here
my_Vertices.append(Vertex(vertices[i], temp_list1, temp_list2))
temp_list1 = []
temp_list2 = []

In [5]:
print("Vertices with their parent set and costs are:")
for v in my_Vertices:
    print("Vertex is:  ", v.vertex, " Parent set is: ", v.parent, " Cost is: ", v.parent_cost)

Vertices with their parent set and costs are:
Vertex is:   1  Parent set is:  ['{}', '{3}', '{4}', '{5}']  Cost is:  [153.466, 96.093, 97.913, 99.835]
Vertex is:   2  Parent set is:  ['{}', '{3}', '{4}', '{5}']  Cost is:  [141.023, 122.446, 121.576, 123.398]
Vertex is:   3  Parent set is:  ['{}', '{1}', '{2}', '{1,2}', '{4}', '{5}']  Cost is:  [169.482, 112.109, 150.906, 107.516, 51.681, 41.775]
Vertex is:   4  Parent set is:  ['{}', '{1}', '{2}', '{1,2}', '{3}', '{5}']  Cost is:  [169.482, 113.929, 150.036, 108.982, 51.681, 36.188]
Vertex is:   5  Parent set is:  ['{}', '{1}', '{2}', '{1,2}', '{3}', '{4}']  Cost is:  [169.802, 116.171, 152.178, 111.473, 42.096, 36.508]


In [6]:
# The function to calculate minimum cost of a given ordering
def minimum_cost(ordering):
    cost = 0
    # empty_flag used for checking disconnected
    empty_flag = False
    for element in ordering:
        # Getting index of the element
        x = ordering.index(element)
        # The previous elements in the ordering
        previous_elements = copy.deepcopy(ordering[0:x])
        ele = int(element)
        for v in vertices:
            if v == element:
                # Getting parents and costs of the element
                parents = copy.deepcopy(my_Vertices[ele - 1].parent)
                costs = copy.deepcopy(my_Vertices[ele - 1].parent_cost)
                while True:
                    minimum = min(costs)
                    index = costs.index(minimum)
                    # Splitting parents, this is done for the nodes that have more than one parent
                    parent = copy.deepcopy(parents[index])
                    parent = copy.deepcopy(parent.split(","))

                    # When parent is not {} we have to retrieve parents like {1} and {1,2}
                    if parent[0] != "{}":
                        for i in range(len(parent)):
                            parent[i] = parent[i].strip("{}")

                    # empty_flag used to check for disjoint graph
                    # If disjoint appears then it will have to select some other minimum cost
                    # If {} gives parent cost then simply return making sure that if exists its previous had {} too
                    if parent[0] == "{}" and empty_flag is False:
                        cost += minimum
                        break
                    # Checks if the parent set that gives minimum is consistent with previous_elements
                    flag = all(x in previous_elements for x in parent)
                    if flag == True:
                        empty_flag = True
                        cost += minimum
                        break
                    else:
                        costs[index] = 10000000
    return cost

In [7]:
# The TreeNode class
class TreeNode:
    def __init__(self, data):
        self.data = copy.deepcopy(data)
        self.children = []

    def ret_data(self):
        return self.data

    def addChild(self, child):
        self.children.append(TreeNode(child))

    def return_child(self):
        return self.children

In [8]:
# Class of Tree
class Tree:
    def __init__(self):
        self.root = None
        # variables for dfs algorithm
        self.minimum_cost_dfs = 1000000.0
        self.minimum_order_dfs = []

    def addRoot(self, data):
        self.root = TreeNode(data)

    # The function makes permutations by swapping and adds in the list
    def add_children(self, node, total_vertices, index):
        if total_vertices == 0:
            return
        for x in range(total_vertices):
            temp_lis = copy.deepcopy(node.ret_data())
            temp_lis[index], temp_lis[x] = temp_lis[x], temp_lis[index]
            node.addChild(temp_lis)

        x = 0
        child_list = node.return_child()
        while x < len(child_list):
            self.add_children(child_list[x], total_vertices - 1, index + 1)
            x += 1

    def createTree(self, no_vertices):
        self.add_children(self.root, no_vertices, 0)

    # BFS Function
    def bfs(self):
        visited = []
        fringe = []
        fringe.append(self.root)
        minimum_node_cost = 1000000000
        minimum_ordering = []
        while fringe:  # Creating loop to visit each node
            m = fringe.pop(0)
            cost = minimum_cost(m.ret_data())
            if cost < minimum_node_cost:
                minimum_node_cost = cost
                minimum_ordering = m.ret_data()

            for neighbour in m.return_child():
                if neighbour not in visited:
                    visited.append(neighbour)
                    fringe.append(neighbour)
        print(f"Ordering from bfs {minimum_ordering} gives the minimum cost i.e. {minimum_node_cost}")

    # Functions for dfs
    def dfs_algorithm(self, visited, node):  # function for dfs
        if node not in visited:
            cost = minimum_cost(node.ret_data())
            if cost < float(self.minimum_cost_dfs):
                self.minimum_cost_dfs = cost
                self.minimum_order_dfs = node.ret_data()
            visited.append(node)
            for neighbour in node.return_child():
                self.dfs_algorithm(visited, neighbour)
            return

    def dfs(self):
        self.dfs_algorithm([], self.root)
        print(f"Ordering from dfs {self.minimum_order_dfs} gives the minimum cost i.e. {self.minimum_cost_dfs}")


In [9]:
# Main Function
if __name__ == "__main__":
    ordering = copy.deepcopy(vertices)
    tree = Tree()
    tree.addRoot(ordering)
    tree.createTree(len(vertices))
    tree.bfs()
    tree.dfs()


Ordering from bfs ['4', '2', '5', '3', '1'] gives the minimum cost i.e. 465.43399999999997
Ordering from dfs ['5', '4', '3', '1', '2'] gives the minimum cost i.e. 465.43399999999997
